In [24]:
from z3 import *

x = Real('x')
y = Real('y')
z = Real('z')

s = Solver()

s.add(3*x + 2*y - z == 0)
s.add(2*x - 2*y + 4*z == 0)
s.add(-x + 0.5*y - z == 0)

s.check()
if s.check() == 'sat':
    print (s.model())
else:
    print('unsat')


unsat


In [4]:
import pysat.solvers
from pysat.card import CardEnc as encode

width = 18
height = 19

lower_bound = 1
upper_bound = 1
w,h = max(width, height), min(width, height)
while h > 0:
    upper_bound += 1
    w,h = max(w-h, h), min(w-h, h)
    
while upper_bound - lower_bound > 1:
    n = (upper_bound + lower_bound)//2
    print(f'Lower bound = {lower_bound}, upper bound = {upper_bound}, solving for {n} or fewer squares')
    
    variable_number = 0
    int_to_variable = {}
    clauses = []
    grid = [[[] for x in range(width)] for y in range(height)]
    
    for x in range(width):
        for y in range(height):
            for size in range(1,1+min(width-x,height-y)):
                variable_number += 1
                # Variable encodes the existence of a square with given coordinates and size
                int_to_variable[variable_number] = (x,y,size)
                for x1 in range(size):
                    for y1 in range(size):
                        grid[y+y1][x+x1].append(variable_number)

    for x in range(width):
        for y in range(height):
            # One square covers each cell
            clauses += encode.equals(lits=grid[y][x], bound = 1, top_id = variable_number, encoding = 8).clauses
            variable_number = max(abs(literal) for clause in clauses for literal in clause)
    
    # Exactly n squares total
    clauses += encode.atmost(lits=list(int_to_variable), bound = n, top_id = variable_number, encoding = 8).clauses

    print(clauses[0])
    solver = pysat.solvers.Cadical()
    for clause in clauses:
        solver.add_clause(clause)
    sat = solver.solve()

    if sat:
        model = solver.get_model()
        solution = [int_to_variable[m] for m in int_to_variable if m in model]
        print(f'Solution found with {len(solution)} squares:',solution)
        upper_bound = len(solution)
    else:
        print(f'No solution exists with {n} or fewer squares')
        lower_bound = n
        
print(f'Least number of squares is {upper_bound}')

Lower bound = 1, upper bound = 20, solving for 10 or fewer squares
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Solution found with 10 squares: [(0, 0, 6), (0, 6, 3), (0, 9, 5), (0, 14, 5), (3, 6, 1), (3, 7, 2), (4, 6, 1), (5, 6, 13), (6, 0, 6), (12, 0, 6)]
Lower bound = 1, upper bound = 10, solving for 5 or fewer squares


KeyboardInterrupt: ignored

In [2]:
!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 3.4 MB 49.1 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=19305799e1aab34adfd68becd29b861ff0bc8ba26ea7c4b2232f69ead2f7533d
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious
